We can also apply our exact diagonalization methods to other models, such as the XXZ model:

$$ H = \sum_i \Delta\sigma_i^z \sigma_{i+1}^z + J(\sigma_i^x\sigma_{i+1}^x + \sigma_i^y\sigma_{i+1}^y) $$

As before, we want to have just a single parameter, so we look at $H/J$, with the free parameter $g = \Delta/J$.

We will then do the following:

1) Find the $2^N\times 2^N$ matrices for the ZZ, XX, and YY terms

2) Find eigenstates and eigenvalues for a range of g

3) Find expectation values in the ground state for various operators

4) Look at dynamics in the model: if we start with a particular spin configuration, what happens to it over time?

As before, $N>10$ may run out your computer's memory, so be careful!

# Preliminaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Sx = np.array( [[0,1],[1,0]] )
Sz = np.array( [[1,0],[0,-1]] )
Sy = np.array( [[0,-1j],[1j,0]] )
Id = np.array( [[1,0],[0,1]] )

# Finding the matrices

## ZZ matrix

First I demonstrate three different ways of finding the matrix for the $\sigma^z \sigma^z$ terms, same as for the Ising model

**Method 1: Kronecker product**

In [ ]:
def ZZ(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sz Sz terms, NOT including the factor of g
    
    Computed using Kronecker product method
    """
    ZZ = np.zeros( (2**N,2**N) , dtype = int) # Create matrix of all zeros
    # Add Sz Sz terms one by one, from left to right
    for i in range(N-1):
        operators = [Id]*i + [Sz,Sz] + [Id]*(N-2-i) # A list of all the operators in this term 
        term = operators[0]
        for op in operators[1:]: # iterate through all elements after the first one, which was used to start the list
            term = np.kron(term, op)
        # Add result for this term to the matrix
        ZZ += term
    return ZZ

**Method 2: Column by column**

In [ ]:
def d2b(i,N):
    b = np.array( [ (i//2**(N-1-j))%2 for j in range(N) ] )
    return b

def b2d(b):
    N = len(b)
    d = np.sum( [ b[i]*2**(N-1-i) for i in range(N) ] )
    return d

def ZZ_v2(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sz Sz terms, NOT including the factor of g
    
    Computed using Column-by-column method
    """
    ZZ = np.zeros( (2**N,2**N) , dtype = int)
    for col in range(2**N):
        b = d2b(col, N)
        for j in range(N-1):
            coeff = (-1)**( b[j] + b[j+1] )              
            row = col
            ZZ[row, col] += coeff
    return ZZ

In [ ]:
ZZ_v2(3)

**Method 3: Use knowledge that the operator is diagonal**

In [ ]:
def ZZ_v3(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sz Sz terms, NOT including the factor of g
    
    Computed using knowledge that operator is diagonal
    """
    ZZ_diag = np.zeros(2**N, dtype = int)
    for col in range(2**N):
        b = d2b(col,N)
        val = 0
        for j in range(N-1):
            val += (-1)**(b[j] + b[j+1])
        ZZ_diag[col] = val
    ZZ = np.diag(ZZ_diag)
    return ZZ

In [ ]:
ZZ_v3(3)

**Test that all three agree for some larger values of $N$**

In [ ]:
N = 8
print(np.linalg.norm(ZZ(N)-ZZ_v2(N)))
print(np.linalg.norm(ZZ(N)-ZZ_v3(N)))

## XX matrix, YY matrix

Next are the XX and YY matrices, which I construt in two ways

**Method 1: Kronecker product**

In [ ]:
# Version 1: Kronecker product

def XX(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sx Sx terms
    
    Computed using Kronecker product method
    """
    # Your code here
    XX = np.zeros( (2**N,2**N) , dtype = int) # Create matrix of all zeros
    # Add Sz Sz terms one by one, from left to right
    for i in range(N-1):
        operators = [Id]*i + [Sx,Sx] + [Id]*(N-2-i) # A list of all the operators in this term 
        term = operators[0]
        for op in operators[1:]: # iterate through all elements after the first one, which was used to start the list
            term = np.kron(term, op)
        # Add result for this term to the matrix
        XX += term
    return XX

def YY(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sy Sy terms
    
    Computed using Kronecker product method
    """
    # Your code here
    YY = np.zeros( (2**N,2**N) , dtype = complex) # Create matrix of all zeros
    # Add Sz Sz terms one by one, from left to right
    for i in range(N-1):
        operators = [Id]*i + [Sy,Sy] + [Id]*(N-2-i) # A list of all the operators in this term 
        term = operators[0]
        for op in operators[1:]: # iterate through all elements after the first one, which was used to start the list
            term = np.kron(term, op)
        # Add result for this term to the matrix
        YY += term
    if (np.abs(np.imag(YY)) > 10**-12).any(): 
        raise ValueError("erroneous imaginary part")
    YY = np.real(YY)
    return YY

In [ ]:
print(XX(2))
print(YY(2))

**Method 2: Column by column**

In [ ]:
# Version 2: Column-by-column

def XX_v2(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sx Sx terms
    
    Computed using Column-by-column method
    """
    # Your code here
    XX = np.zeros( (2**N,2**N), dtype = int )
    for col in range(2**N):
        b = d2b(col, N)
        for j in range(N-1):
            coeff = 1
            new_b = b.copy()
            new_b[j] = 1-new_b[j] # Flip the spin on site j
            new_b[j+1] = 1-new_b[j+1]
            row = b2d(new_b)
            XX[row, col] += coeff
    return XX

def YY_v2(N):
    """
    Input: N is the number of sites
    Output: Matrix for sum of Sx Sx terms
    
    Computed using Column-by-column method
    """
    # Your code here
    YY = np.zeros( (2**N,2**N), dtype = int )
    for col in range(2**N):
        b = d2b(col, N)
        for j in range(N-1):
            coeff = (-1)**(1+b[j]+b[j+1])
            new_b = b.copy()
            new_b[j] = 1-new_b[j] # Flip the spin on site j
            new_b[j+1] = 1-new_b[j+1]
            row = b2d(new_b)
            YY[row, col] += coeff
    return YY

In [ ]:
print(XX_v2(2))
print(YY_v2(2))

** Compare the methods for large N**

In [ ]:
N=6
print(np.linalg.norm(XX(N)-XX_v2(N)))
print(np.linalg.norm(YY(N)-YY_v2(N)))

# Solving the model

We now use these matrices to solve the XXZ model!

**Energy spectrum**

The first step is to just look at the energy eigenspectrum.  I provide the program here.  There are parameters for you to select:

```N```: the number of sites

```num_pts```: how many values of the parameter ```g``` to use, in the interval from 0 to 2

```show_lowest```: if the value is ```False``` all $2^N$ eigenvalues will be plotted.  If it is a positive integer $n$, the lowest $n$ eigenvalues will be shown (or $2^N$ if that is less than $n$).

```gs_at_0```: if ```False``` the actual energies will be plotted; if ```True```, for each $g$ the ground state energy will be subtracted from all eigenvalues, so the plot is of "relative energy" above the ground state.

```per_site```: if ```True```, all energy eigenvalues will be divided by ```N``` to give the intensive quantity energy/site 

I recommend exploring these different options

In [ ]:
# Parameters
N = 2
num_pts = 21
show_lowest = False
gs_at_0 = False
per_site = False

# Generate X and ZZ matrices using the fastest methods
XX = XX_v2(N) 
YY = YY_v2(N)
ZZ = ZZ_v3(N)

# Function to get H from these stored matrices
def get_H(g):
    return g*ZZ + XX + YY

# Values of g to use in calculating/plotting
gs = np.linspace(0,2,num_pts)

# Create an empty table to store the results:
energies = np.zeros( (2**N,num_pts) )
energies[:,:] = np.nan 

for g_index, g in enumerate(gs): 
    H = get_H(g)
    e, v = np.linalg.eigh(H) 
    inds = np.argsort(e)
    e = e[inds]
    v = v[:, inds]
    
    energies[:,g_index] = e
    
if per_site:
    energies /= N
if gs_at_0:
    for i in range(2**N):
        energies[i] -= energies[0]

f, a = plt.subplots()
max_ind = 2**N if not show_lowest else show_lowest
for row in range(max_ind): 
    a.scatter(gs, energies[row])
a.set_xlabel('g', fontsize = 16);
a.set_ylabel('E', fontsize = 16);

Note that ```g=1``` with ```N=2``` is actually just the $S_\text{tot}^2$ operator for two spin-1/2 systems.  See the earlier supplemental exercise for the identification of the the four energy levels with spin 0 and spin 1. 

**Expectation values**

Next we want to look at some physical properties, as measured by expectation values and correlation functions in the ground state.  This model has a great deal of symmetry, since it is invariant if you flip all spins in the x, y, or z basis.  As a result, any expectation value of $\sigma^x$, $\sigma^y$, or $\sigma^z$ will be 0.  (The special point $g=1$ has even more symmetry, and in fact the component of spin in any direction, $(\sigma^x,\sigma^y,\sigma^z)\cdot\hat{n}$ will have zero expectation value.  This is called $SO(3)$ symmetry.)  

So we will focus on the correlation functions of the three spin components on the first and last sites, $\langle \sigma^x_0 \sigma^x_{N-1}\rangle$, $\langle \sigma^y_0 \sigma^y_{N-1}\rangle$, and $\langle \sigma^z_0 \sigma^z_{N-1}\rangle$.

In [ ]:
# Parameters
N = 5
num_pts = 21

# Generate X and ZZ matrices using the fastest methods
XX = XX_v2(N) 
YY = YY_v2(N)
ZZ = ZZ_v3(N)

# Function to get H from these stored matrices
def get_H(g):
    return g*ZZ + XX + YY

# Values of g to use in calculating/plotting
gs = np.linspace(0,2,num_pts)

# Generate a matrix for each operator whose expectation value we want to measure.  
#  (Hint: you already basically have the one for x!) 

def get_ZZ_end_site_matrix(N):
    """
    For N sites, return the matrix for the operator \sigma^z_0 \otimes Id \otimes ... \otimes Id \otimes \sigma^z_{N-1}
    """
    # Your code here
    ZZ_ends = np.zeros( 2**N, dtype = int)
    for i in range(2**N):
        b = d2b(i, N)
        ZZ_ends[i] = (-1)**(b[0]+b[-1])
    return np.diag(ZZ_ends)

def get_XX_end_site_matrix(N):
    """
    For N sites, return the matrix for the operator \sigma^x_0 \otimes Id \otimes ... \otimes Id \otimes \sigma^x_{N-1}
    """
    # Your code here
    XX_ends = np.zeros( (2**N,2**N), dtype = int)
    for col in range(2**N):
        b = d2b(col, N)
        b[0] = 1-b[0]
        b[N-1] = 1-b[N-1]
        row = b2d(b)
        XX_ends[row,col] = 1
    return XX_ends

def get_YY_end_site_matrix(N):
    """
    For N sites, return the matrix for the operator \sigma^y_0 \otimes Id \otimes ... \otimes Id \otimes \sigma^y_{N-1}
    """
    # Your code here
    YY_ends = np.zeros( (2**N,2**N), dtype = int)
    for col in range(2**N):
        b = d2b(col, N)
        coeff = (-1)**(1+b[0]+b[N-1])
        b[0] = 1-b[0]
        b[N-1] = 1-b[N-1]
        row = b2d(b)
        YY_ends[row,col] = coeff
    return YY_ends

ZZ_ends = get_ZZ_end_site_matrix(N)
XX_ends = get_XX_end_site_matrix(N)
YY_ends = get_YY_end_site_matrix(N)

# For storing the results
YY_ends_vals = np.zeros(num_pts)
ZZ_ends_vals = np.zeros(num_pts)
XX_ends_vals = np.zeros(num_pts)

# Do the calculations
for g_index, g in enumerate(gs):
    H = get_H(g)
    e, v = np.linalg.eigh(H)
    indices = np.argsort(e)
    e = e[indices] 
    v = v[:, indices] 
    
    ZZ_ends_vals[g_index] = v[:,0].conj().transpose() @ ZZ_ends @ v[:,0]
    XX_ends_vals[g_index] = v[:,0].conj().transpose() @ XX_ends @ v[:,0]
    YY_ends_vals[g_index] = v[:,0].conj().transpose() @ YY_ends @ v[:,0]
    
# Make plots
for (values, label) in [(XX_ends_vals, '<XX>'), (YY_ends_vals, '<YY>'), (ZZ_ends_vals, '<ZZ>')]:
    f,a = plt.subplots()
    a.scatter(gs, values)
    a.set_xlabel('g', fontsize = 16);
    a.set_ylabel(label, fontsize = 16);

# Comparing different N

In condensed matter physics, the goal is to study very large systems, with many particles (on the order of $10^{23}$).  So when we study small systems, as we've done going to just 10-ish sites in this case, it is important to think about what would happen if we made $N$ larger.  To do this, we plot the results for many values of $N$ together.  

Here we make four figures comparing the results as we change $N$:
- The energy gap between the ground state and the first excited state
- The energy gap between the first and second excited states
- The expectation value of the average of $\sigma^x$
- The correlation function of $\langle \sigma^z \sigma^z\rangle$ on the first and last sites

In [ ]:
Ns = range(2,9) # Includes 2 through 8
num_pts = 21
gs = np.linspace(0,2,num_pts)

# Arrays to store results for all N and g
first_E_gap = np.zeros( (len(Ns),num_pts) )
second_E_gap = np.zeros( (len(Ns),num_pts) )
ZZ_ends_vals = np.zeros( (len(Ns),num_pts) )
XX_ends_vals = np.zeros( (len(Ns),num_pts) )
YY_ends_vals = np.zeros( (len(Ns),num_pts) )

# Loop through values of N
for N_idx, N in enumerate(Ns):
    XX = XX_v2(N)
    YY = YY_v2(N)
    ZZ = ZZ_v3(N)
    XX_ends = get_XX_end_site_matrix(N)
    YY_ends = get_YY_end_site_matrix(N)
    ZZ_ends = get_ZZ_end_site_matrix(N)
    
    for g_index, g in enumerate(gs):
        H = g*ZZ + XX + YY
        e, v = np.linalg.eigh(H)
        indices = np.argsort(e)
        e = e[indices] 
        v = v[:, indices] 

        first_E_gap[N_idx, g_index] = e[1]-e[0]
        second_E_gap[N_idx, g_index] = e[2]-e[1]
        ZZ_ends_vals[N_idx, g_index] = v[:,0].conj().transpose() @ ZZ_ends @ v[:,0]
        XX_ends_vals[N_idx, g_index] = v[:,0].conj().transpose() @ XX_ends @ v[:,0]
        YY_ends_vals[N_idx, g_index] = v[:,0].conj().transpose() @ YY_ends @ v[:,0]
        
# Make plots
for (values, label) in [(first_E_gap,'$\Delta_1$'),(second_E_gap,'$\Delta_2$'),(ZZ_ends_vals, '<ZZ>'),(XX_ends_vals, '<XX>'),(YY_ends_vals, '<YY>')]:
    f,a = plt.subplots()
    for N_idx, N in enumerate(Ns):
        a.plot(gs, values[N_idx], label = N) # 'label' option will let us make a legend
    a.set_xlabel('g', fontsize = 16);
    a.set_ylabel(label, fontsize = 16);
    a.legend()
    a.axhline(0, c='gray', zorder = 0, alpha=0.5) # Makes a horizontal line to make results easier to understand
    if label in ['<X>','<ZZ>']: a.axhline(1, c='gray', zorder=0, alpha=0.5)

From these calculations you can get a sense of how som behaviors seem to be converged already at $N\approx 10$ while others are not.  To get a better sense, you can make plots where $g$ is fixed and you plot the values versus $1/N$.  Since we stored all the results above, this doesn't require any extra calculation.  Here is an example:

In [ ]:
# Which g to plot
g_val = 1.5

# Find the index in the output array for this g (will give an error if this g is not in the set that was computed)
g_idx = np.where(np.abs(gs - g_val) < 10**-10)

# Plot vs 1/N
f,a = plt.subplots()
a.scatter(1/np.array(Ns), first_E_gap[:,g_idx])
a.set_xlim(left=0);
a.set_ylim(bottom=0);

Here we see the pretty interesting behavior that the degeneracy of the levels depends on whether the system size is even or odd!  This is because the ground state always has the lowest possible total spin, and when the system has an even number of sites there is just one symmetry sector with the lowest spin (0), but with an odd number there are two (1/2 and -1/2), hence the two degenerate ground states.

# Dynamics

In general, if we know the eigenvalues and eigenstates of a Hamiltonian, we can find the time evolution of any given initial state, in the following way:

1. Decompose the state as a superposition of energy eigenvectors:
$$|\psi\rangle = \sum_i c_i|v_i\rangle$$

2. Put in the time evolution of each eigenstate:
$$|\psi(t)\rangle = \sum_i c_i e^{i E t/\hbar}|v_i\rangle$$

Here since we've messed with the units a whole bunch and have a unitless "energy" we can just make the opposite transformations to $t/\hbar$ and get a unitless "time" $\tau$ and look at $|\psi(\tau)\rangle$:
$$|\psi(t)\rangle = \sum_i c_i e^{i E \tau}|v_i\rangle$$

In the following we will do this for a 10-site system with an initial state that has spin up on site 0 and spin in the $+\hat{x}$ direction on all other sites.


In [ ]:
N=10
g = 1.0
H = g*ZZ_v3(N) + XX_v2(N) + YY_v2(N)

e,v = np.linalg.eigh(H)

In [ ]:
def get_z(state, site):
    """
    returns expectation value of sigma^z on site 'site' in the given state, which is a vector of coefficients in the 
    usual basis
    """
    total = 0
    for row in range(2**N):
        b = d2b(row, N)
        coeff = (-1)**(b[site])
        total += np.abs(state[row])**2 * coeff
    return total

def get_z_all_sites(state):
    """
    returns an np.array of length N containing the expectation value of sigma^z on each site
    """
    totals = np.zeros(N)
    for row in range(2**N):
        b = d2b(row, N)
        coeffs = (-1)**b
        totals += np.abs(state[row])**2 * coeffs
    return totals

In [ ]:
v0 = np.ones(2**(N-1))/np.sqrt(2**(N-1)) # N-1 spins in +x
v0 = np.kron(np.array([1,0]), v0) # First spin in +z, rest in +x

# Initial sigma^z expectation values to test:
plt.scatter(range(N),get_z_all_sites(v0))

basis_state_coeffs = v.conj().transpose() @ v0
#Time evolution
def get_v(t):
    coeffs = basis_state_coeffs * np.exp(-1j*e*t)
    vt = v @ coeffs
    return vt

In [ ]:
t_max = 10
num_ts = 201

ts = np.linspace(0,t_max,num_ts)
sz_vals = np.zeros( (num_ts, N) )

for t_idx, t in enumerate(ts):
    vt = get_v(t)
    sz_vals[t_idx] = get_z_all_sites(vt)
    
f,a = plt.subplots()
c = a.imshow(sz_vals, aspect = N/num_ts)
a.set_ylabel('t')
a.set_xlabel('site')
f.colorbar(c);

In [ ]:
total_sz_vals = np.sum(sz_vals, axis=1)
g,b = plt.subplots()
b.scatter(ts, total_sz_vals)
b.set_xlabel('t')
b.set_ylabel('Total <Sz>');

In [ ]:
f,a = plt.subplots()
for t_ind in range(8):
    a.plot(range(N),sz_vals[t_ind], marker = '.', linestyle='-')